# Efficient Yelp API Calls 
- Christina Reeder
- 2 Mar 2023

In [7]:
!pip install yelpapi

In [8]:
import json
from yelpapi import YelpAPI

## Create and Save Yelp API Key

In [9]:
with open('C:/Users/James/Documents/DataEnrichment/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [11]:
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api